In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
abc = pd.read_csv('Data/InsuranceCompanyABC.csv')
cde = pd.read_csv('Data/InvestmentBankCDE.csv')
efg = pd.read_csv('Data/RetailBankEFG.csv')
answers = pd.read_csv('Data/ANSWERS.csv')

In [3]:
abc.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial
0,71504.00,42.99,4428.51,NE,1.00,T,T,T,T
1,59751.00,26.37,4489.93,CO,0.00,T,T,T,T


In [4]:
cde.head(2)

,ID,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto
0,71504.00,F,F,F,F,F,F,F
1,59751.00,F,F,F,F,F,F,F


In [5]:
efg.head(2)

,ID,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,71504.00,T,T,T,T,T,F,F,T,F
1,59751.00,T,T,T,T,T,F,F,F,T


In [6]:
df1 = pd.merge(cde, efg, on='ID')
data = pd.merge(abc, df1, on='ID')
data.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos
0,71504.00,42.99,4428.51,NE,1.00,T,T,T,T,F,F,F,F,F,F,F,T,T,T,T,T,F,F,T,F
1,59751.00,26.37,4489.93,CO,0.00,T,T,T,T,F,F,F,F,F,F,F,T,T,T,T,T,F,F,F,T


In [7]:
answers.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.00,40.69,8474.69,CO,1.00,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.00,56.69,7996.35,SE,0.00,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# exploration = pd.DataFrame({
#     'column': data.columns, 'type': data.dtypes, 'NA #': data.isna().sum(), 'NA %': round((data.isna().sum() / data.shape[0]) * 100, 2)
# })
# exploration.sort_values('NA %', ascending=False)

In [9]:
onehot = lambda x: 1 if x == 'T' else 0

In [10]:
for cols in data.columns[5::]:
    data[cols] = data[cols].apply(onehot)

In [152]:
# Building the model
frq_items = apriori(data.iloc[:, 5::], min_support=0.1, use_colnames=True,)

frq_items
 
# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="confidence", min_threshold=0.8)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
47,"(seguro auto, seguro vida Emp, seguro vida PF)",(Seguro Residencial),0.20,0.33,0.20,1.00,3.00,0.13,361.93
100,"(Investimento LCA, Investimento Fundos_cambiai...",(Investimento Fundos_commodities),0.12,0.35,0.12,0.99,2.87,0.08,78.58
48,"(seguro auto, seguro vida Emp, Seguro Residenc...",(seguro vida PF),0.20,0.25,0.20,0.99,3.96,0.15,78.79
46,"(Seguro Residencial, seguro vida Emp, seguro v...",(seguro auto),0.20,0.28,0.20,0.99,3.54,0.14,56.47
99,"(Investimento LCA, Investimento Fundos_commodi...",(Investimento LCI),0.12,0.34,0.12,0.99,2.93,0.08,50.94
...,...,...,...,...,...,...,...,...,...
79,"(Investimento Fundos_cambiais, Investimento CD...",(Investimento LCI),0.13,0.34,0.11,0.81,2.41,0.06,3.50
4,(Emprestimo _educacao),(Investimento Fundos),0.16,0.34,0.13,0.80,2.36,0.07,3.35
33,"(Investimento Fundos, Investimento LCI)",(Investimento CDB),0.16,0.49,0.13,0.80,1.62,0.05,2.57
41,"(Emprestimo _viagem, Investimento Fundos)",(Emprestimo _educacao),0.15,0.16,0.12,0.80,5.01,0.09,4.23


In [153]:
rules['lenght'] = rules['antecedents'].apply(lambda x: len(x))

In [154]:
rules = rules[(rules['lenght'] > 1) & (rules['confidence'] >= 0.8)].reset_index(drop=True)

In [155]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,lenght
0,"(seguro auto, seguro vida Emp, seguro vida PF)",(Seguro Residencial),0.20,0.33,0.20,1.00,3.00,0.13,361.93,3
1,"(Investimento LCA, Investimento Fundos_cambiai...",(Investimento Fundos_commodities),0.12,0.35,0.12,0.99,2.87,0.08,78.58,4
2,"(seguro auto, seguro vida Emp, Seguro Residenc...",(seguro vida PF),0.20,0.25,0.20,0.99,3.96,0.15,78.79,3
3,"(Seguro Residencial, seguro vida Emp, seguro v...",(seguro auto),0.20,0.28,0.20,0.99,3.54,0.14,56.47,3
4,"(Investimento LCA, Investimento Fundos_commodi...",(Investimento LCI),0.12,0.34,0.12,0.99,2.93,0.08,50.94,4
...,...,...,...,...,...,...,...,...,...,...
107,"(Investimento Fundos_commodities, Investimento...","(Investimento Fundos_cambiais, Investimento LCI)",0.16,0.20,0.13,0.81,4.01,0.10,4.25,2
108,"(Investimento CDB, seguro vida PF)",(seguro auto),0.12,0.28,0.10,0.81,2.91,0.07,3.83,2
109,"(Investimento Fundos_cambiais, Investimento CD...",(Investimento LCI),0.13,0.34,0.11,0.81,2.41,0.06,3.50,3
110,"(Investimento Fundos, Investimento LCI)",(Investimento CDB),0.16,0.49,0.13,0.80,1.62,0.05,2.57,2


In [58]:
# rules.groupby(['antecedents', 'consequents']).agg({'confidence': 'max'}).sort_values('confidence', ascending=False)

In [20]:
# for row1, row2 in zip(rules.itertuples(index=False), answers.itertuples(index=False)):
#     print(row1, row2)
#     break

In [19]:
answers.head(2)

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3
0,1004.00,40.69,8474.69,CO,1.00,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN
1,1028.00,56.69,7996.35,SE,0.00,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
index = []
aux = []
services = []
for val in range(data.shape[0]):
    # index.append(answers.index[val])
    for cols in data.columns[5::]:
        if answers[cols][val] == 'T':
            aux.append(cols)
        else: continue
    services.append(aux)
    aux = []

answers['services'] = services

In [186]:
# answers['services'] = answers['services'].apply(lambda x: ", ".join(x))
# rules['antecedents'] = rules['antecedents'].apply(lambda x: ", ".join(x))
answers['services'] = answers['services'].apply(lambda x: set(x))
# rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x))

In [149]:
answers['services'].head(2), rules['antecedents'].head(2)

(0    [Investimento Fundos_commodities, Investimento...
 1                       [Investimento Fundos_cambiais]
 Name: services, dtype: object,
 0       [seguro auto, seguro vida Emp, seguro vida PF]
 1    [Investimento LCA, Investimento Fundos_cambiai...
 Name: antecedents, dtype: object)

In [188]:
consequents = []
i = 0
for j in range(answers.shape[0]):
    if rules['antecedents'][i] & answers['services'][j]:
        consequents.append(rules['consequents'][i])
    else :
        consequents.append(None)
    if i == rules.shape[0] - 1:
        i = 0
    i += 1

In [ ]:
consequents1 = []
consequents1 = []
consequents1 = []
i = 0
for j in range(answers.shape[0]):
    if rules['antecedents'][i] & answers['services'][j]:
        consequents.append(rules['consequents'][i])
    else :
        consequents.append(None)
    if i == rules.shape[0] - 1:
        i = 0
    i += 1

In [180]:
answers['consequents'] = consequents

In [183]:
answers

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3,services,consequents
0,1004.00,40.69,8474.69,CO,1.00,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Investimento Fundos Multimercado, Investiment...",None
1,1028.00,56.69,7996.35,SE,0.00,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,{Investimento Fundos_cambiais},(Investimento Fundos_commodities)
2,1034.00,49.41,8624.33,NE,0.00,T,T,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Seguro Residencial, seguro vida Emp, seguro a...",(seguro vida PF)
3,1035.00,22.76,6885.20,S,1.00,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Investimento LCA, Seguro Residencial}",(seguro auto)
4,1039.00,22.06,7749.97,NE,1.00,T,T,T,T,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Seguro Residencial, seguro vida Emp, seguro a...",(Investimento LCI)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,100932.00,28.66,8804.51,S,1.00,F,F,F,F,F,T,T,F,T,F,F,F,F,F,T,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Investimento Fundos_commodities, Emprestimo _...",None
13726,100962.00,20.50,6809.90,SE,0.00,F,F,F,F,F,F,F,F,F,T,F,F,F,F,T,F,F,T,T,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Investimento Fundos Multimercado, Emprestimo ...",None
13727,100963.00,21.07,6201.48,SE,1.00,F,F,F,F,F,T,F,F,F,F,T,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Investimento Fundos_commodities, Investimento...",None
13728,100978.00,43.48,8130.08,N,0.00,T,T,T,T,F,F,T,F,F,T,T,F,F,F,F,F,T,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,"{Investimento Tesouro Direto, Seguro Residenci...","(seguro auto, seguro vida PF)"


In [181]:
len(consequents)

13730

In [120]:
len(answers.services.unique())

5713

In [121]:
rules['antecedents']

0           seguro auto, seguro vida Emp, seguro vida PF
1      Investimento LCA, Investimento Fundos_cambiais...
2       seguro auto, seguro vida Emp, Seguro Residencial
3      Seguro Residencial, seguro vida Emp, seguro vi...
4      Investimento LCA, Investimento Fundos_commodit...
                             ...                        
107    Investimento Fundos_commodities, Investimento LCA
108                     Investimento CDB, seguro vida PF
109    Investimento Fundos_cambiais, Investimento CDB...
110                Investimento Fundos, Investimento LCI
111              Emprestimo _viagem, Investimento Fundos
Name: antecedents, Length: 112, dtype: object

In [116]:
answers.loc[answers['services'].isin(rules['antecedents'])]

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3,services
153,2330.00,35.74,7711.68,S,1.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,T,NaN,NaN,NaN,NaN,NaN,NaN,"Emprestimo _viagem, Investimento Fundos"
179,2533.00,46.69,6767.31,SE,1.00,F,F,F,F,T,T,F,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"Investimento Fundos_cambiais, Investimento Fun..."
311,3774.00,29.07,8312.87,SE,0.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,T,NaN,NaN,NaN,NaN,NaN,NaN,"Emprestimo _viagem, Investimento Fundos"
313,3789.00,50.47,8432.96,NE,1.00,F,F,F,F,F,F,T,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"Investimento LCI, Investimento LCA"
430,4779.00,19.61,7487.92,CO,1.00,F,F,F,F,F,F,T,T,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"Investimento LCI, Investimento LCA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13706,100860.00,40.37,6911.59,S,1.00,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"seguro auto, seguro vida Emp"
13707,100860.00,40.37,6911.59,S,1.00,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"seguro auto, seguro vida Emp"
13708,100860.00,40.37,6911.59,S,1.00,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"seguro auto, seguro vida Emp"
13709,100860.00,40.37,6911.59,S,1.00,T,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,"seguro auto, seguro vida Emp"


In [112]:
answers.loc[answers['services'].isin(rules['antecedents'])]

SystemError: <built-in method view of numpy.ndarray object at 0x7f567898e440> returned a result with an error set

In [111]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,lenght
0,"(seguro auto, seguro vida Emp, seguro vida PF)",(Seguro Residencial),0.20,0.33,0.20,1.00,3.00,0.13,361.93,3
1,"(Investimento LCA, Investimento Fundos_cambiai...",(Investimento Fundos_commodities),0.12,0.35,0.12,0.99,2.87,0.08,78.58,4
2,"(seguro auto, seguro vida Emp, Seguro Residenc...",(seguro vida PF),0.20,0.25,0.20,0.99,3.96,0.15,78.79,3
3,"(Seguro Residencial, seguro vida Emp, seguro v...",(seguro auto),0.20,0.28,0.20,0.99,3.54,0.14,56.47,3
4,"(Investimento LCA, Investimento Fundos_commodi...",(Investimento LCI),0.12,0.34,0.12,0.99,2.93,0.08,50.94,4
...,...,...,...,...,...,...,...,...,...,...
107,"(Investimento Fundos_commodities, Investimento...","(Investimento Fundos_cambiais, Investimento LCI)",0.16,0.20,0.13,0.81,4.01,0.10,4.25,2
108,"(Investimento CDB, seguro vida PF)",(seguro auto),0.12,0.28,0.10,0.81,2.91,0.07,3.83,2
109,"(Investimento Fundos_cambiais, Investimento CD...",(Investimento LCI),0.13,0.34,0.11,0.81,2.41,0.06,3.50,3
110,"(Investimento Fundos, Investimento LCI)",(Investimento CDB),0.16,0.49,0.13,0.80,1.62,0.05,2.57,2


In [102]:

if set(rules['antecedents'][107]) & set(answers['services'][0]) != set():
    print(rules['consequents'][107])


frozenset({'Investimento Fundos_cambiais', 'Investimento LCI'})


In [ ]:
# print(set(answers['services'][0]).intersection(set(rules['antecedents'][107])))

In [56]:
answers[answers['services'].str.lower().isin([x.lower() for x in l])]

,ID,Idade,Renda,Regiao,Genero,seguro auto,seguro vida Emp,seguro vida PF,Seguro Residencial,Investimento Fundos_cambiais,Investimento Fundos_commodities,Investimento LCI,Investimento LCA,Investimento Poupanca,Investimento Fundos Multimercado,Investimento Tesouro Direto,Financiamento Casa,Financiamento Carro,Emprestimo _pessoal,Emprestimo _consignado,Emprestimo _limite_especial,Emprestimo _educacao,Emprestimo _viagem,Investimento CDB,Investimento Fundos,RECOMMENDATION_1,CONFIDENCE_1,RECOMMENDATION_2,CONFIDENCE_2,RECOMMENDATION_3,CONFIDENCE_3,services
101,1904.00,24.93,7477.07,NE,1.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB
191,2704.00,40.75,8797.11,SE,0.00,F,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento Fundos_commodities
192,2712.00,19.05,7774.48,SE,1.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB
193,2717.00,47.65,7434.31,CO,0.00,F,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento Fundos_commodities
222,2899.00,43.66,6348.94,NE,0.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13014,96038.00,28.27,6697.64,SE,0.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB
13164,97304.00,42.73,8361.33,NE,0.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB
13228,97589.00,32.91,7222.53,S,1.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB
13229,97589.00,33.91,7400.11,N,1.00,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,NaN,NaN,NaN,NaN,NaN,NaN,Investimento CDB


In [22]:
for row1, row2 in zip(data.itertuples(index=False), services.itertuples(index=False)):
    print(row1)
    break

AttributeError: 'list' object has no attribute 'itertuples'

In [ ]:
# set(services[0]) & set(rules['antecedents'][0])

set()

In [ ]:
# # recomendations = pd.DataFrame({'index': index, 'services': services})
# confidence = []
# conf = []
# index = []
# for serv in range(len(services)):
#     index.append(answers.index[serv])
#     for ant in range(rules.shape[0]):
#         if set(services[serv]) & set(rules['antecedents'][ant]):
#             conf.append(rules['confidence'][ant])
#         else: continue
#     confidence.append(conf)
#     conf = []

In [ ]:
len(confidence)

13730